In [ ]:
using LinearAlgebra
using Statistics
using Random
using Printf

function create_data()
    Random.seed!(1)
    n = 1000
    
    area = rand(20:200, n)
    rooms = rand(1:4, n)
    age = rand(0:50, n)
    floor = rand(1:22, n)
    distance = rand(1.0:30.0, n)
    
    X = hcat(ones(n), area, rooms, age, floor, distance)
    
    true_coeffs = [1500000, 120000, 800000, -25000, 50000, -75000]
    
    y = X * true_coeffs + randn(n) * 500000
    
    return X, y, ["Intercept", "Area", "Rooms", "Age", "Floor", "Distance"]
end

function split_data(X, y, test_size=0.2)
    
    n = size(X, 1)
    indices = shuffle(1:n)
    test_rows = Int(round(n * test_size))
    
    train_indices = indices[1:end-test_rows]
    test_indices = indices[end-test_rows+1:end]
    
    X_train, y_train = X[train_indices, :], y[train_indices]
    X_test, y_test = X[test_indices, :], y[test_indices]
    
    return X_train, y_train, X_test, y_test
end

function linear_regression(X, y)
    coefficients = X \ y
    return coefficients
end

function evaluate_model(X_test, y_test, coefficients)
    predictions = X_test * coefficients
    mse = mean((predictions .- y_test).^2)
    rmse = sqrt(mse)
    mae = mean(abs.(predictions .- y_test))
    r_squared = 1 - sum((y_test .- predictions).^2) / sum((y_test .- mean(y_test)).^2)

    println("Метрики качества:")
    @printf "MSE: %.2f\n" mse
    @printf "RMSE: %.2f руб\n" rmse
    @printf "MAE: %.2f руб\n" mae  
    @printf "R²: %.4f\n" r_squared
    
    return predictions, (mse=mse, rmse=rmse, mae=mae, r_squared=r_squared)
end

function main()
    println("="^50)
    println("ПРОЕКТ: ПРОГНОЗИРОВАНИЕ ЦЕН НА НЕДВИЖИМОСТЬ")
    println("="^50)
    
    X, y, names = create_data()
    
    X_train, y_train, X_test, y_test = split_data(X, y)
    println()
    
    coefficients = linear_regression(X_train, y_train)
    println()
    
    println("Коэффициенты модели:")
    for (i, name) in enumerate(names)
        println("$name: ", round(coefficients[i], digits=2))
    end
    println()
    
    predictions, metrics = evaluate_model(X_test, y_test, coefficients)
    println()
    
    println("Пример прогнозирования для новой квартиры:")
    println("")
    new_apartment = [2, 38, 1, 13, 18, 20]
    predicted_price = dot(new_apartment, coefficients)
    println("Характеристики квартиры:")
    println("Площадь: $(new_apartment[2]) м²")
    println("Количество комнат: $(new_apartment[3])")
    println("Возраст квартиры: $(new_apartment[4]) лет")
    println("Этаж квартиры: $(new_apartment[5])")
    println("Расстояние до центра: $(new_apartment[6]) км")
    println("")

    @printf "Предсказанная цена: %.2f рублей\n" predicted_price
    
    return coefficients, metrics
end

if abspath(PROGRAM_FILE) == @__FILE__
    coefficients, metrics = main()
end

# Комментарии по коду
## Функция create_data
Функция генерирует данные из 1000 наблюдений для обучения модели.

Характеристики данных:
- Площадь квартиры
- Количество комнат
- Год постройки квартиры
- Этаж квартиры
- Расстояние до центра города 

Эти характеристики заносятся в матрицу Х.

Также есть истинные коэффициенты квартиры:
- Базовая стоимость квартиры
	+ 1.500.000 рублей (1) - для регионов
	+ 2.250.000 рублей (1,5) - для Санкт-Петербурга
	+ 3.000.000 рублей (2) - для Москвы
- Площадь квартиры (+120.000 рублей / м²)
- Количество комнат (+800.000 рублей / комната)
- Год постройки квартиры (-25.000 рублей / год)
- Этаж квартиры (+50.000 рублей / этаж)
- Расстояние до центра города (-75.000 рублей / км)

В итоге в переменной y мы получаем вектор цен для 1.000 квартир

## Функция split_data
Функция разделяет данные на обучающую (80%) и тестовую (20%) выборки. Для того, чтобы не было упорядоченности данных массив из 1000 чисел перемешивается.

## Функция linear_regression
Функция находит коэффициенты линейной модели, которые лучше всего описывают зависимость между характеристиками и ценой.

## Функция evaluate_model
Функция оценивает точность модели, используя различные метрики.

Используемые метрики:
- MSE (Среднеквадратичное отклонение)
- RMSE (Среднеквадратическая ошибка)
- MAE (Средняя абсолютная ошибка)
- R² (Коэффициент детерминации)

## Функция main
Основная функция программы, в которой выводится пример прогнозирования цены для новой квартиры.